In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
project_data = pd.read_excel('../data/TEIS-NSS Project Data 10-2022.xlsx', 
                             sheet_name=['ECO with Exit21-22', 'Elig Timeline Rpt 2018-2022'])


In [ ]:
eco_exit = project_data.get('ECO with Exit21-22')
elig_timeline = project_data.get('Elig Timeline Rpt 2018-2022')
print(eco_exit)
print(elig_timeline)

In [ ]:
eco_exit

In [ ]:
elig_timeline

In [ ]:
type(elig_timeline)

#### Dropped all columns that did not appear to be relevant to analysis

In [ ]:
elig_timeline = elig_timeline.drop(elig_timeline.columns[[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33]],
                                   axis=1)
elig_timeline

#### Separated Init Elig Cat into 3 DFs based on condition listed

In [ ]:
diag_cond = elig_timeline[elig_timeline['Init. Elig. Category'] == 'Diagnosed Condition']

dev_eval = elig_timeline[elig_timeline['Init. Elig. Category'] == 'Developmental Evaluation']

diag_dev_comb = elig_timeline[elig_timeline['Init. Elig. Category'] == 'Diagnosed Condition, Developmental Evaluation']

In [ ]:
##determine unique exit reasons and total number of each occurrence
exit_reason = elig_timeline['Exit Reason'].value_counts()
exit_reason

In [ ]:
exit_reason = exit_reason.to_frame()

##need to figure out how to fix column names. df.colums= gives me an error saying it's a list
exit_reason

In [ ]:
##dropped unused columns
eco_exit = eco_exit.drop(eco_exit.columns[[16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,
                                          41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59]], axis=1)
eco_exit

In [ ]:
## standardize column names
elig_timeline = elig_timeline.rename(columns={'District':'district',
                                              'Child ID':'child_id',
                                              'Child Status':'child_status',
                                              'Child Phase':'child_phase',
                                              'Init. Elig. Category':'eligibility_category',
                                              'Init. Elig. Det. Date': 'date_elig_determined',
                                              'Exit Date':'exit_date',
                                              'Exit Reason':'exit_reason'})
elig_timeline.head()

In [ ]:
eco_exit = eco_exit.rename(columns={'DISTRICT':'district',
                                    'CHILD_ID':'child_id',
                                    'ECO_Entry_DATE':'eco_entry_date',
                                    'Ent SOCIAL_SCALE':'ent_social',
                                    'Ent KNOWLEDGE_SCALE':'ent_knowledge',
                                    'Ent APPROPRIATE_ACTION_SCALE':'ent_approp_action',
                                    'ECO_Exit_DATE':'eco_exit_date',
                                    'Exit SOCIAL_SCALE':'exit_social',
                                    'Exit KNOWLEDGE_SCALE':'exit_knowledge',
                                    'Exit APPROPRIATE_ACTION_SCALE':'exit_approp_action',
                                    'ExitIS_SOCIAL_PROGRESS':'exit_social_progress',
                                    'ExitIS_KNOWLEDGE_PROGRESS':'exit_knowledge_progress',
                                    'ExitIS_APPROPRIATE_ACTION_PROGRESS':'exit_approp_act_progress',
                                    'Initial IFSP Date':'initial_ifsp_date',
                                    'Exit Reason':'exit_reason',
                                    'Exit Date':'exit_date'})
                                            
eco_exit.head()

In [ ]:
type(eco_exit['child_id'])

In [ ]:
##remove last 4 rows of DF that are all NaN and converted child_id from float to int

eco_exit = eco_exit.iloc[:-4 , :]
eco_exit['child_id'] = eco_exit['child_id'].astype('int')
type(eco_exit['child_id'])

In [ ]:
eco_exit

In [ ]:
##datasets joined for analysis

eco_elig_combined = pd.merge(elig_timeline, eco_exit, on=['child_id', 'district'], how='inner')
eco_elig_combined

In [ ]:
##pulled diagnosed condition out to average

diag_cond_comb = eco_elig_combined[eco_elig_combined['eligibility_category'] == 'Diagnosed Condition']
diag_cond_comb

In [ ]:
##drop nan values

diag_cond_comb = diag_cond_comb.dropna(axis=0)
diag_cond_comb

In [ ]:
##verify nan values dropped

diag_cond_comb.isna().sum()

In [ ]:
diag_cond_comb= diag_cond_comb.drop(diag_cond_comb.columns[[2, 5, 6, 7]], axis=1)
diag_cond_comb

In [ ]:
diag_cond_comb['ent_avg'] = diag_cond_comb.loc[:, ['ent_social', 'ent_knowledge', 'ent_approp_action']].mean(axis=1)
diag_cond_comb['exit_avg'] = diag_cond_comb.loc[:, ['exit_social', 'exit_knowledge', 'exit_approp_action']].mean(axis=1)
diag_cond_comb